In [12]:
import os
import shutil
import warnings
import urllib.request as request
from contextlib import closing

import cupy

import dask_cudf
import dask_ml
import dask

from cuml.manifold import UMAP as cuUMAP
from cuml.dask.decomposition import PCA as cuDaskPCA
from cuml.dask.cluster import KMeans as cuDaskKMeans
from cuml.dask.manifold import UMAP as cuDaskUMAP

from dask.distributed import Client, LocalCluster
from dask_cuda import initialize, LocalCUDACluster
from dask_cuda.local_cuda_cluster import cuda_visible_devices

from bokeh.io.export import export_png
from bokeh.plotting import figure
from bokeh.models.tickers import FixedTicker
from bokeh.io import output_notebook, push_notebook, show

from nvidia.cheminformatics.chembldata import ChEmblData

warnings.filterwarnings('ignore', 'Expected ')
warnings.simplefilter('ignore')

output_notebook()

Loading BokehJS ...

### Configurations and settings

In [2]:
# Please add or remove device ids that can be used
# CUDA_VISIBLE_DEVICES=[0]
CUDA_VISIBLE_DEVICES = cuda_visible_devices(0).split(',')

pca_comps = 64
n_clusters = 6
n_neighbors=100
num_mols=500000

enable_tcp_over_ucx = True
enable_nvlink = False
enable_infiniband = False

COLORS = ["#406278", "#e32636", "#9966cc", "#cd9575", "#915c83", "#008000",
          "#ff9966", "#848482", "#8a2be2", "#de5d83", "#800020", "#e97451",
          "#5f9ea0", "#36454f", "#008b8b", "#e9692c", "#f0b98d", "#ef9708",
          "#0fcfc0", "#9cded6", "#d5eae7", "#f3e1eb", "#f6c4e1", "#f79cd4"]
FINGER_PRINT_FILES = 'filter_*.h5'

### Functions

In [3]:
def show_cluster_plot(ldcudf, title='UMAP'):
    """
    Draws a scatter plots from output of UMAP.
    """
    umap_fig = figure(title=title, width=800, output_backend="webgl")
    clusters = ldcudf['cluster'].unique().compute().values_host

    for cluster in clusters:
        query = 'cluster == %s' % (cluster)

        cdf = ldcudf.query(query)
        cdf = cdf.compute()

        if cdf.shape[0] == 0:
            continue

        x_array = cupy.fromDlpack(cdf['x'].to_dlpack())
        y_array = cupy.fromDlpack(cdf['y'].to_dlpack())

        umap_fig.circle(x_array.get(),
                        y_array.get(),
                        size=2,
                        color=COLORS[cluster],
                        alpha=0.5, legend = 'Cluster ' + str(cluster))

    umap_fig.legend.location = 'top_right'
    umap_fig.legend.title = 'Clusters'
    
    umap_fig_handle = show(umap_fig, notebook_handle=True)
    push_notebook(handle=umap_fig_handle)
    

def recluster(tcdf):
    """
    Recluster on a dataset
    """
    tcdf = tcdf.persist()

    print('Reclustering...')
    tcdf = tcdf.drop(['x', 'y', 'cluster'], axis=1)

    print('Computing KMEANS...')
    kmeans_float = cuDaskKMeans(client=client, n_clusters=n_clusters)
    kmeans_float.fit(tcdf)
    kmeans_labels = kmeans_float.predict(tcdf)

    print('Computing UMAP...')
    local_model = cuUMAP()
    X_train = tcdf.compute()
    local_model.fit(X_train)

    umap_model = cuDaskUMAP(local_model,
                            n_neighbors=n_neighbors,
                            a=1.0,
                            b=1.0,
                            learning_rate=1.0,
                            client=client)
    Xt = umap_model.transform(tcdf)

    # Add back the column required for plotting and to correlating data
    # between re-clustering
    tcdf['x'] = Xt[0]
    tcdf['y'] = Xt[1]
    tcdf['cluster'] = kmeans_labels
    
    return tcdf

# Download ChEMBL database

In [4]:
data_dir = "/data/db"
db_file = os.path.join(data_dir, 'chembl_27.db')

if not os.path.exists(db_file):
    print('Downloading ChEMBL db...')

    os.makedirs(data_dir, exist_ok=True)
    with closing(request.urlopen('ftp://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/chembl_27_sqlite.tar.gz')) as r:
        with open(db_file, 'wb') as f:
            shutil.copyfileobj(r, f)

    print('Download completed')
else:
    print('Reusing available ChEMBL db at', db_file)

Reusing available ChEMBL db at /data/db/chembl_27.db


In [5]:
cluster = LocalCUDACluster(protocol="ucx",
                           dashboard_address=':9001',
                           # TODO: automate visible device list
                           CUDA_VISIBLE_DEVICES=CUDA_VISIBLE_DEVICES,
                           enable_tcp_over_ucx=enable_tcp_over_ucx,
                           enable_nvlink=enable_nvlink,
                           enable_infiniband=enable_infiniband)
client = Client(cluster)
client

Client Scheduler: ucx://127.0.0.1:35727 Dashboard: http://127.0.0.1:9001/status,Cluster Workers: 3 Cores: 3 Memory: 33.58 GB


# Generate fingerprint from ChEMBL

The 4 in ECFP4 corresponds to the diameter of the atom environments considered, while the Morgan fingerprints take a radius parameter. So a Morgan fingerprint with radius=2 is roughly equivalent to ECFP4 and FCFP4.

In [15]:
%%time
cache_directory = '/data/fp'

if cache_directory is None:
    chem_data = ChEmblData(db_file=db_file)
    ddf = chem_data.fetch_all_props(num_recs=num_mols)
else:
    hdf_path = os.path.join(cache_directory, FINGER_PRINT_FILES)
    ddf = dask.dataframe.read_hdf(hdf_path, 'fingerprints')

    if num_mols > 0:
        ddf = ddf.head(num_mols, compute=False, npartitions=-1)

dcudf = dask_cudf.from_dask_dataframe(ddf)
dcudf = dcudf.persist()

# Reduce
pca = cuDaskPCA(client=client, n_components=pca_comps)
dcudf = pca.fit_transform(dcudf)

CPU times: user 13.7 s, sys: 762 ms, total: 14.5 s
Wall time: 50.6 s


# Clustering

In [16]:
%%time

kmeans_float = cuDaskKMeans(client=client, n_clusters=n_clusters)
kmeans_float.fit(dcudf)
kmeans_labels = kmeans_float.predict(dcudf)

CPU times: user 131 ms, sys: 31.2 ms, total: 162 ms
Wall time: 1.1 s


In [17]:
%%time
local_model = cuUMAP()
X_train = dcudf.compute()
local_model.fit(X_train)

umap_model = cuDaskUMAP(local_model,
                        n_neighbors=n_neighbors,
                        a=1.0,
                        b=1.0,
                        learning_rate=1.0,
                        client=client)
Xt = umap_model.transform(dcudf)

dcudf['x'] = Xt[0]
dcudf['y'] = Xt[1]
dcudf['cluster'] = kmeans_labels

dcudf.head(5)

CPU times: user 8.94 s, sys: 2.71 s, total: 11.6 s
Wall time: 27 s


,0,1,2,3,4,5,6,7,8,9,...,57,58,59,60,61,62,63,x,y,cluster
0,-0.841311,-0.532717,0.593586,-0.300685,0.386239,-0.083094,0.298180,1.064561,-1.051220,-0.058497,...,0.300635,0.430448,-0.351631,-0.566681,0.649597,0.013187,-0.165357,-2.418525,-0.272940,1
1,-1.194588,0.077112,0.088081,0.026718,0.621446,0.009780,0.857635,0.222406,-0.638917,0.194643,...,0.104665,0.152239,-0.078329,-0.694553,0.572819,-0.093369,0.085992,-1.553411,-1.048763,1
2,-0.826879,-0.318295,-0.032415,-0.191886,0.852638,1.226381,-0.008093,1.237501,-0.505571,0.200043,...,-0.268780,0.442836,-0.179016,-0.692021,0.720313,-0.035165,0.230449,-2.480140,-0.250356,1
3,-1.188946,-0.013169,-0.099914,-0.391794,0.353066,-0.021429,0.666960,-0.002886,-0.674282,0.119216,...,0.335285,-0.116625,-0.185822,-0.392319,0.870164,0.345550,-0.206593,-0.777536,-1.352810,1
4,-1.083622,-0.216352,0.220709,-0.075452,1.361422,0.428857,0.113669,1.287101,-1.162962,-0.145004,...,-0.111000,0.512250,-0.148317,-0.992919,0.631888,0.200106,-0.113605,-2.400434,-0.254294,1


In [ ]:
%%time
show_cluster_plot(dcudf)

# Re-clustering

In [ ]:
%%time
# Filter data before reclustering
cluster = 1                                         # <- PLEASE CHANGE CLUSTER AS REQUIRED
query = 'cluster == ' + str(cluster)
tcdf = dcudf.query(query)

show_cluster_plot(tcdf, title='UMAP - cluster ' + str(cluster))
tcdf = recluster(tcdf)
print('Plotting UMAP...')
show_cluster_plot(tcdf, title='UMAP - cluster ' + str(cluster))